# Retrieval augmented generation (RAG)

In [1]:
# Download a file
import wget

file_path = "my_document.pdf"
doc_url = "https://www.legifrance.gouv.fr/download/file/rxcTl0H4YnnzLkMLiP4x15qORfLSKk_h8QsSb2xnJ8Y=/JOE_TEXTE"

wget.download(doc_url, out=file_path)

-1 / unknown

'my_document.pdf'

In [2]:
# Request configuration
import requests

base_url = "http://localhost:8080/v1"
api_key = "albert" # enter any value if no api_key is setup in config.ini file

session = requests.session()
session.headers = {"Authorization": f"Bearer {api_key}"}  # skip headers if no api_key is setup in config.ini file

In [3]:
# Upload a file
import os

collection = "leo"
model = "BAAI/bge-m3"
params = {"collection": collection, "model": model} 

files = {'files': (os.path.basename(file_path), open(file_path, 'rb'), "application/pdf")}
response = session.post(f"{base_url}/files", params=params , files=files)

response.json()

{'object': 'list',
 'data': [{'object': 'upload',
   'id': 'a7fdd4e0-f9b5-4b2d-9bde-0a42d097e7cf',
   'filename': 'my_document.pdf',
   'status': 'success'}]}

In [4]:
# Retrieve the file ID for RAG
response = session.get(f"{base_url}/files/{collection}")

response.json()

{'object': 'list',
 'data': [{'object': 'file',
   'id': 'a7fdd4e0-f9b5-4b2d-9bde-0a42d097e7cf',
   'bytes': 133606,
   'filename': 'my_document.pdf',
   'created_at': 1720606506}]}

In [5]:
file_id = response.json()["data"][0]["id"]

In [6]:
# Get tools
response = session.get(f"{base_url}/tools")

response.json()

{'object': 'list',
 'data': [{'id': 'BaseRAG',
   'description': 'Base RAG, basic retrival augmented generation.\n\n    Args:\n        embeddings_model (str): OpenAI embeddings model\n        collection (List[Optional[str]]): Collection names. Defaults to "user" parameter.\n        file_ids (Optional[List[str]], optional): List of file ids for user collections (after upload files). Defaults to None.\n        k (int, optional): Top K per collection (max: 6). Defaults to 4.\n        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.',
   'object': 'tool'},
  {'id': 'UseFiles',
   'description': 'Fill your prompt with file contents. Your prompt must contain "{files}" placeholder.\n\n    Args:\n        file_ids (List[str]): List of file ids.',
   'object': 'tool'}]}

In [7]:
# Display tools parameters
print(response.json()["data"][0]["description"])

Base RAG, basic retrival augmented generation.

    Args:
        embeddings_model (str): OpenAI embeddings model
        collection (List[Optional[str]]): Collection names. Defaults to "user" parameter.
        file_ids (Optional[List[str]], optional): List of file ids for user collections (after upload files). Defaults to None.
        k (int, optional): Top K per collection (max: 6). Defaults to 4.
        prompt_template (Optional[str], optional): Prompt template. Defaults to DEFAULT_PROMPT_TEMPLATE.


In [8]:
# OpenAI client configuration
from openai import OpenAI

client = OpenAI(base_url=base_url, api_key=api_key)

In [9]:
# Chat completions
user = "leo"

data = {
    "model": "AgentPublic/llama3-instruct-8b",
    "messages": [{"role": "user", "content": "Qui est Ulrich Tan ?"}],
    "stream": False,
    "n": 1,
    "user": user,
    "tools": [
        {
            "function": {
                "name": "BaseRAG",
                "parameters": {
                    "embeddings_model": model, 
                    "collections": [collection],
                    "k": 2
                }
            },
            "type": "function"
        }
    ]
}

response = client.chat.completions.create(**data)
print(response.choices[0].message.content)

Selon les documents, Ulrich Tan est le chef du pôle Datamin du département "Etalab".


In [10]:
# View chat history
chat_id = response.id

response = session.get(url=f"{base_url}/chat/history/{user}/{chat_id}")
print(response.json()["messages"][0]["content"])

Qui est Ulrich Tan ?
